# Consigna del desafío 2

**1**.  Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.

**2**.  Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).

**3**.  Graficarlos.

**4**. Obtener conclusiones.

## Importamos librerias

In [52]:
!pip install gensim

In [53]:
import os
import platform
import multiprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models.callbacks import CallbackAny2Vec



## Importamos el dataset

In [54]:
# Descargar la carpeta de dataset

if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


## 1 Creamos los vectores

In [55]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/eminem.txt', sep='/n', header=None)
df.head()

/tmp/ipython-input-3634877419.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,"Look, I was gonna go easy on you and not to hu..."
1,But I'm only going to get this one chance
2,"Something's wrong, I can feel it (Six minutes,..."
3,"Just a feeling I've got, like something's abou..."
4,"If that means, what I think it means, we're in..."


In [56]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 6812


In [57]:
sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [58]:
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [59]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,
                     window=2,
                     vector_size=300,
                     negative=20,
                     workers=1,
                     sg=1)

# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [60]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de docs en el corpus: 6812
Cantidad de words distintas en el corpus: 1347


In [61]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 506556.96875
Loss after epoch 1: 369829.53125
Loss after epoch 2: 344129.125
Loss after epoch 3: 314718.375
Loss after epoch 4: 310399.0
Loss after epoch 5: 295839.5
Loss after epoch 6: 262763.5
Loss after epoch 7: 258724.5
Loss after epoch 8: 254935.0
Loss after epoch 9: 252422.0
Loss after epoch 10: 249889.75
Loss after epoch 11: 246938.0
Loss after epoch 12: 243465.0
Loss after epoch 13: 241613.5
Loss after epoch 14: 229063.75
Loss after epoch 15: 226176.0
Loss after epoch 16: 225148.5
Loss after epoch 17: 223591.5
Loss after epoch 18: 222550.5
Loss after epoch 19: 222643.0


(785273, 1296760)

## 2 Probamos terminos de interes

In [62]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["god"], topn=10)

[('food', 0.7980446815490723),
 ('stamps', 0.7704891562461853),
 ('damn', 0.7608792781829834),
 ('ooh', 0.7356470823287964),
 ('swear', 0.7144322395324707),
 ('thank', 0.7053040862083435),
 ('yea', 0.6808665990829468),
 ('ah', 0.6683138608932495),
 ('sent', 0.6661420464515686),
 ('acid', 0.656876802444458)]

In [63]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('his', -0.013269864954054356),
 ('hands', -0.042220115661621094),
 ('an', -0.04660608619451523),
 ('over', -0.10607542842626572),
 ('their', -0.11157377809286118),
 ('hold', -0.12165422737598419),
 ('slap', -0.12283757328987122),
 ('box', -0.12992389500141144),
 ('night', -0.13708920776844025),
 ('or', -0.14021563529968262)]

### Conclusion

Al entrenar un modelo Word2Vec con las letras de Eminem se puede ver cómo las palabras no se agrupan por su significado “general” sino por el uso particular que el artista les da en sus canciones. Por ejemplo, al consultar por la palabra god aparecen términos relacionados más con expresiones de énfasis, emociones intensas o situaciones extremas que con un sentido religioso, lo que refleja el estilo crudo y directo de su lenguaje. En el caso de love, al buscar las diez palabras menos relacionadas, se observa que queda muy aislada dentro del vocabulario y que sus conexiones son débiles y con términos más bien funcionales o narrativos, lo cual sugiere que no ocupa un lugar relevante en sus letras. Esto muestra que los embeddings capturan la forma en que un autor utiliza las palabras dentro de su propio universo semántico y que, cuando el corpus es específico como en este caso, el modelo refleja más la manera personal de expresarse que el significado universal de los términos.

## 3 Graficos

In [64]:
def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [65]:
# Graficar los embedddings en 2D
vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=int(len(w2v_model.wv.index_to_key)*0.5)
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [66]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

## Conclusiones generales

Lo que muestran los gráficos es una representación visual del espacio semántico que el modelo fue capaz de construir a partir de las letras de Eminem. Al reducir la dimensionalidad de los vectores con técnicas como t-SNE, las palabras quedan distribuidas en un plano o en un espacio 3D según las relaciones que el modelo aprendió, y eso permite ver de manera intuitiva cómo se agrupan o se alejan entre sí según su contexto de uso. Lo interesante es que en lugar de un significado universal, lo que aparece es la manera particular en que Eminem conecta esas palabras en sus letras, de modo que los clústeres que se forman reflejan su estilo y los temas más recurrentes en su música. En definitiva, los embeddings no solo sirven para cálculos matemáticos de similitud, sino también para visualizar de una forma clara cómo un corpus específico organiza y da sentido a su propio vocabulario bajo su propio contexto de uso.